In [1]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [3]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250830_Squalane 평가\csv파일"

# find Peak 설정
height = 0.05  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)s
num_samples = 10000 # 최종 줄이고자 하는 행 수

In [4]:
# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=100)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=100)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices]

        # 새 파일 이름 및 저장 경로
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

hexa_000.csv → Peak count: 7274
hexa_010.csv → Peak count: 6088
hexa_020.csv → Peak count: 6068
hexa_030.csv → Peak count: 5716
hexa_040.csv → Peak count: 5877
hexa_050.csv → Peak count: 5429
hexa_100.csv → Peak count: 5227
hexa_150.csv → Peak count: 5426
hexa_200.csv → Peak count: 5685
hotsq_000.csv → Peak count: 5357
hotsq_010.csv → Peak count: 5607
hotsq_020.csv → Peak count: 5604
hotsq_030.csv → Peak count: 5679
hotsq_040.csv → Peak count: 5562
hotsq_050.csv → Peak count: 5377
hotsq_100.csv → Peak count: 5557
hotsq_150.csv → Peak count: 5822
hotsq_200.csv → Peak count: 5595
hotsq_250.csv → Peak count: 6078
OTS_000.csv → Peak count: 5587
OTS_010.csv → Peak count: 5512
OTS_020.csv → Peak count: 5760
OTS_030.csv → Peak count: 6112
OTS_040.csv → Peak count: 5572
OTS_050.csv → Peak count: 5369
OTS_100.csv → Peak count: 5310
OTS_150.csv → Peak count: 5513
OTS_200.csv → Peak count: 5089
pdms_020.csv → Peak count: 5092
pdms_030.csv → Peak count: 5245
pdms_040.csv → Peak count: 5245
pdms_05